In [ ]:
!pip install -q anthropic

In [ ]:
import os
try:
    from google.colab import userdata
    os.environ["ANTHROPIC_API_KEY"] = userdata.get("ANTHROPIC_API_KEY")
except ImportError:
    pass  # Not running in Colab; uses env var already set


# Exercise 5: Executing a Tool

In the previous exercise, Claude asked us to call a tool. Now we'll complete the round-trip: execute the tool and send the result back.

By the end, you'll understand:
- How to append the assistant's tool-use response to the message list
- How to construct a `tool_result` message
- The full request/response cycle for tool use

## Step 1: Setup (same as previous exercise)

In [ ]:
import anthropic

client = anthropic.Anthropic()

tools = [
    {
        "name": "get_weather",
        "description": "Get the current weather in a given location",
        "input_schema": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. San Francisco, CA",
                }
            },
            "required": ["location"],
        },
    }
]

## Step 2: Get the tool-use response

In [ ]:
messages = [{"role": "user", "content": "What is the weather in Tokyo?"}]

message = client.messages.create(
    model="claude-haiku-4-5-20251001",
    max_tokens=1024,
    messages=messages,
    tools=tools,
)

print(f"stop_reason: {message.stop_reason}")
print(f"content: {message.content}")

## Step 3: Append the assistant's response

Just like in multi-turn chat, we need to add the assistant's response to the message list before continuing.

In [ ]:
messages.append({"role": "assistant", "content": message.content})

## Step 4: Find the tool_use block and "execute" the tool

In a real application, you'd call an actual weather API here. For now, we return a hardcoded result.

In [ ]:
tool_use_block = next(block for block in message.content if block.type == "tool_use")

print(f"Claude wants to call: {tool_use_block.name}")
print(f"With arguments: {tool_use_block.input}")
print(f"Tool use ID: {tool_use_block.id}")

# "Execute" the tool (hardcoded for now)
tool_result = "65 degrees"

## Step 5: Send the tool result back

We send the result as a `tool_result` content block inside a `user` message. The `tool_use_id` must match the `id` from the tool_use block — this is how Claude knows which tool call this result is for.

In [ ]:
messages.append(
    {
        "role": "user",
        "content": [
            {
                "type": "tool_result",
                "tool_use_id": tool_use_block.id,
                "content": tool_result,
            }
        ],
    }
)

final_response = client.messages.create(
    model="claude-haiku-4-5-20251001",
    max_tokens=1024,
    messages=messages,
    tools=tools,
)

print(final_response.content[0].text)

## Step 6: Review the full message flow

Let's look at the complete conversation to see the three-step pattern:
1. **User** asks a question
2. **Assistant** responds with a `tool_use` block
3. **User** sends back a `tool_result`
4. **Assistant** gives the final natural language answer

In [ ]:
messages.append({"role": "assistant", "content": final_response.content})

for msg in messages:
    print(f"--- {msg['role']} ---")
    print(f"{msg['content']}\n")

## Key takeaway

The tool-use cycle is:

**You → Claude:** "What's the weather?" (+ tool definitions)  
**Claude → You:** "Please call `get_weather` with `{"location": "Tokyo"}`"  
**You → Claude:** "Here's the result: 65 degrees"  
**Claude → You:** "The weather in Tokyo is 65 degrees."

In the next exercise, we'll automate this into a loop so it can handle multiple tool calls.